In [89]:
import pandas as pd
import plotly.express as px
import re

In [90]:
# Define the results you want to explore
csv_path = "./results/results_cellpdia30_sigma4_dilrad4_dnad_obj_seg_v2_gliaero6_gliathr20.csv" 

# Read the CSV file
df = pd.read_csv(csv_path)

# Convert the index into a column
df.reset_index(inplace=True)

# Extract 'tissue_location'
df['tissue_location'] = df['filename'].str.split('_40X_').str[-1]

# Extract 'staining_id'
df['staining_id'] = df['filename'].str.extract('(\d+)_40X')

# Display the first few rows of the DataFrame
df.head()


,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,%_dna_damage_signal,%_glia+_signal,tissue_location,staining_id
0,0,DSB Iba1 16_40X_CA1,0.292208,1.071429,0.333333,1.045455,42,112,154,207,0.209141,88.633728,CA1,16
1,1,DSB Iba1 16_40X_CA3,0.739884,1.641026,0.785714,1.662791,78,95,173,182,0.511169,96.349525,CA3,16
2,2,DSB Iba1 16_40X_CTX1,0.055556,1.000000,0.183521,1.020833,2,34,36,267,0.130939,13.598442,CTX1,16
3,3,DSB Iba1 16_40X_CTX2,0.501976,1.270000,0.490775,1.266667,100,153,253,271,0.296688,94.930744,CTX2,16
4,4,DSB Iba1 16_40X_CTX3,0.567797,1.264151,0.565041,1.362745,53,65,118,246,0.392723,54.768944,CTX3,16


In [91]:
# Extract analysis parameters from the CSV path
extracted_values = re.findall(r'\d+', csv_path)

# Dynamically assign the extracted values to variables
if len(extracted_values) >= 6:
    cellpose_nuclei_diameter = int(extracted_values[0])
    gaussian_sigma = int(extracted_values[1])
    dilation_radius_nuclei = int(extracted_values[2])
    dna_damage_segmenter_version = int(extracted_values[3])
    glia_nuclei_colocalization_erosion = int(extracted_values[4])
    if "_glia_sem_seg_v" in str(csv_path):
        glia_segmenter = True
    else:
        glia_segmenter = False

if glia_segmenter:
    glia_segmenter_version = int(extracted_values[5])
    glia_channel_threshold = None
    # Dinamically adjust plot titles
    title = f"cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_glia_sem_seg_v{glia_segmenter_version}"
else:
    glia_channel_threshold = int(extracted_values[5])
    glia_segmenter_version = None
    # Dinamically adjust plot titles
    title = f"cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_gliathr{glia_channel_threshold}"
    

# Print the assigned analysis parameters
print(f"Cellpose nuclei diameter: {cellpose_nuclei_diameter}")
print(f"Gaussian sigma: {gaussian_sigma}")
print(f"Dilation radius nuclei: {dilation_radius_nuclei}")
print(f"Dna damage segmenter version: {dna_damage_segmenter_version}")
print(f"Glia erosion: {glia_nuclei_colocalization_erosion}")
print(f"Glia threshold: {glia_channel_threshold}")
print(f"Glia semantic segmentation version: {glia_segmenter_version}")



Cellpose nuclei diameter: 30
Gaussian sigma: 4
Dilation radius nuclei: 4
Dna damage segmenter version: 2
Glia erosion: 6
Glia threshold: 20
Glia semantic segmentation version: None


In [92]:
mouse_id_csv_path = "./mouse_ids_Iba1.csv"

# Read the CSV file
df_mouse_id = pd.read_csv(mouse_id_csv_path, delimiter=";", encoding="UTF-8")

# Display the first few rows of the DataFrame
df_mouse_id.head()

,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,manual_qc
0,34,885,male,APP/PS1,poor,passed
1,53,885,male,APP/PS1,good,passed
2,23,2042,male,APP/PS1,good,passed
3,31,2042,male,APP/PS1,good,passed
4,43,2042,male,APP/PS1,good,passed


In [93]:
df['staining_id'] = pd.to_numeric(df['staining_id'], errors='coerce')
df_mouse_id['staining_id'] = pd.to_numeric(df_mouse_id['staining_id'], errors='coerce')

# Merge both processed_results_df and mouse_id dataframes on staining_id
merged_df = pd.merge(df, df_mouse_id, on="staining_id")

# Display the first few rows of the DataFrame
merged_df.head()

,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,%_dna_damage_signal,%_glia+_signal,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,manual_qc
0,0,DSB Iba1 16_40X_CA1,0.292208,1.071429,0.333333,1.045455,42,112,154,207,0.209141,88.633728,CA1,16,887,male,APP/PS1,poor,failed
1,1,DSB Iba1 16_40X_CA3,0.739884,1.641026,0.785714,1.662791,78,95,173,182,0.511169,96.349525,CA3,16,887,male,APP/PS1,poor,failed
2,2,DSB Iba1 16_40X_CTX1,0.055556,1.000000,0.183521,1.020833,2,34,36,267,0.130939,13.598442,CTX1,16,887,male,APP/PS1,poor,failed
3,3,DSB Iba1 16_40X_CTX2,0.501976,1.270000,0.490775,1.266667,100,153,253,271,0.296688,94.930744,CTX2,16,887,male,APP/PS1,poor,failed
4,4,DSB Iba1 16_40X_CTX3,0.567797,1.264151,0.565041,1.362745,53,65,118,246,0.392723,54.768944,CTX3,16,887,male,APP/PS1,poor,failed


In [94]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_+_dna_damage_glia_nuclei',
                 hover_data=['staining_id','index'], title=f"Number of DNA damage+ Glia+ Nuclei by Tissue Location - {title}")

# Show the plot
fig.show()

In [95]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='avg_dna_damage_foci/glia_+',
                 hover_data=['staining_id'], title=f"Average DNA damage foci in Glia Nuclei by Tissue Location - {title}")

# Show the plot
fig.show()

In [96]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei',
                 hover_data=['staining_id'], title=f"Average DNA damage foci in All Nuclei by Tissue Location - {title}")

# Show the plot
fig.show()

In [97]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_glia_+_nuclei',
                 hover_data=['staining_id','index'], title=f"Nr of glia+ nuclei by Tissue Location - {title}")

# Show the plot
fig.show()

In [98]:
# Create the plot
fig = px.scatter(df, x='tissue_location', y='nr_total_nuclei',
                 hover_data=['staining_id','index'], title=f'Nr of total nuclei by Tissue Location - {title}')

# Show the plot
fig.show()

In [99]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='nr_glia_+_nuclei',
                 hover_data=['tissue_location','index'], title=f'Nr of glia+ nuclei by Sample - {title}')

# Show the plot
fig.show()

In [100]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='%_dna_damage_signal',
                 hover_data=['tissue_location','index'], title=f'Dna damage mask area (QC) - {title}')

# Show the plot
fig.show()

In [101]:
# Create the plot
fig = px.scatter(df, x='staining_id', y='%_glia+_signal',
                 hover_data=['tissue_location','index'], title=f'Glia mask area (QC) - {title}')

# Show the plot
fig.show()

We can observe there is a number of outliers in the glial and dna damage mask detection given the staining is suboptimal in some of the samples. I will filter the data to remove those suboptimal stains and just plot the optimal ones where the automated image analysis offers reliable results.

In [102]:
# Calculate mean area of the image occupied by glia+ signal
glia_mask_area_mean = df['%_glia+_signal'].mean() 

# Calculate mean area of the image occupied by dna_damage_+ signal
dna_damage_mask_area_mean = df['%_dna_damage_signal'].mean() 

# Print extracted values
print(f"Glia_mask_area_%_mean: {glia_mask_area_mean}, Dna_damage_mask_area_%_mean: {dna_damage_mask_area_mean}") 

Glia_mask_area_%_mean: 11.002078888908265, Dna_damage_mask_area_%_mean: 0.4919684122479151


In [103]:
# Define a function to determine staining quality, anything above 3 times the mean value is considered an outlier
def determine_stain_quality(value, mean_value):
    if value < (mean_value + mean_value*3):
        return "optimal"
    else:
        return "suboptimal"

# Check stain quality for glia and create another column storing optimal or suboptimal if qc_passed or not    
merged_df['glia_stain_quality_auto'] = merged_df['%_glia+_signal'].apply(lambda x: determine_stain_quality(x, glia_mask_area_mean))

# Check stain quality for dna_damage and create another column storing optimal or suboptimal if qc_passed or not 
merged_df['dna_damage_stain_quality_auto'] = merged_df['%_dna_damage_signal'].apply(lambda x: determine_stain_quality(x, dna_damage_mask_area_mean))

# Check for both stain qualities and store True qc_passed if both are optimal
merged_df['staining_qc_passed'] = (merged_df['glia_stain_quality_auto'] == 'optimal') & (merged_df['dna_damage_stain_quality_auto'] == 'optimal')

# Group the DataFrame by 'staining_id' and check if all 'staining_qc_passed' values are True, otherwise set them all to False
merged_df['staining_qc_passed'] = merged_df.groupby('staining_id')['staining_qc_passed'].transform('all')

# Now, if all 'staining_qc_passed' values for the same 'staining_id' were True, the column will remain True; otherwise, it will be False

merged_df.head()

,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,...,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,manual_qc,glia_stain_quality_auto,dna_damage_stain_quality_auto,staining_qc_passed
0,0,DSB Iba1 16_40X_CA1,0.292208,1.071429,0.333333,1.045455,42,112,154,207,...,CA1,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
1,1,DSB Iba1 16_40X_CA3,0.739884,1.641026,0.785714,1.662791,78,95,173,182,...,CA3,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
2,2,DSB Iba1 16_40X_CTX1,0.055556,1.000000,0.183521,1.020833,2,34,36,267,...,CTX1,16,887,male,APP/PS1,poor,failed,optimal,optimal,False
3,3,DSB Iba1 16_40X_CTX2,0.501976,1.270000,0.490775,1.266667,100,153,253,271,...,CTX2,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
4,4,DSB Iba1 16_40X_CTX3,0.567797,1.264151,0.565041,1.362745,53,65,118,246,...,CTX3,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False


In [104]:
if glia_segmenter:
    merged_df.to_csv(
        f"./results/qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_glia_sem_seg_v{glia_segmenter_version}.csv",
        index=False,
    )
else:
    merged_df.to_csv(
        f"./results/qc_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}_dilrad{dilation_radius_nuclei}_dnad_obj_seg_v{dna_damage_segmenter_version}_gliaero{glia_nuclei_colocalization_erosion}_gliathr{glia_channel_threshold}.csv",
        index=False,
    )

In [105]:
# Remove data from images with a poor quality stain (auto QC)
auto_filtered_df = merged_df[merged_df['staining_qc_passed'] == True]

In [106]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/glia_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Glia Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - {title}')

# Show the plot
fig.show()

In [107]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/glia_+_damage_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Damaged Glia Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - {title}')

# Show the plot
fig.show()

In [108]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in All Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - {title}')

# Show the plot
fig.show()

In [109]:
# Create the boxplot
fig = px.box(auto_filtered_df, x='tissue_location', y='avg_dna_damage_foci/all_nuclei_damage_+',
             color='genotype', # Different genotypes will be shown in different colors
             title=f'DNA Damage in Damage+ Nuclei by Tissue Location and Genotype (sex-aggregated) - Auto stain QC - {title}')

# Show the plot
fig.show()

Show failed qc dataframe

In [110]:
qc_failed_df = merged_df[merged_df['staining_qc_passed'] == False]

print(f"{qc_failed_df.shape[0]} stains have not passed QC and have been discarded")

qc_failed_list = qc_failed_df['index'].tolist()

qc_failed_df


40 stains have not passed QC and have been discarded


,index,filename,avg_dna_damage_foci/glia_+,avg_dna_damage_foci/glia_+_damage_+,avg_dna_damage_foci/all_nuclei,avg_dna_damage_foci/all_nuclei_damage_+,nr_+_dna_damage_glia_nuclei,nr_-_dna_damage_glia_nuclei,nr_glia_+_nuclei,nr_total_nuclei,...,tissue_location,staining_id,animal_id,sex,genotype,dna_damage_stain_quality_manual,manual_qc,glia_stain_quality_auto,dna_damage_stain_quality_auto,staining_qc_passed
0,0,DSB Iba1 16_40X_CA1,0.292208,1.071429,0.333333,1.045455,42,112,154,207,...,CA1,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
1,1,DSB Iba1 16_40X_CA3,0.739884,1.641026,0.785714,1.662791,78,95,173,182,...,CA3,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
2,2,DSB Iba1 16_40X_CTX1,0.055556,1.000000,0.183521,1.020833,2,34,36,267,...,CTX1,16,887,male,APP/PS1,poor,failed,optimal,optimal,False
3,3,DSB Iba1 16_40X_CTX2,0.501976,1.270000,0.490775,1.266667,100,153,253,271,...,CTX2,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
4,4,DSB Iba1 16_40X_CTX3,0.567797,1.264151,0.565041,1.362745,53,65,118,246,...,CTX3,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
5,5,DSB Iba1 16_40X_DG,0.369668,1.218750,0.377778,1.214286,64,147,211,225,...,DG,16,887,male,APP/PS1,poor,failed,suboptimal,optimal,False
6,6,DSB Iba1 17_40X_CA1,0.181818,1.000000,0.352601,1.109091,2,9,11,173,...,CA1,17,892,female,APP/PS1,poor,failed,optimal,optimal,False
7,7,DSB Iba1 17_40X_CA3,0.450000,1.285714,0.603352,1.440000,7,13,20,179,...,CA3,17,892,female,APP/PS1,poor,failed,optimal,optimal,False
8,8,DSB Iba1 17_40X_CTX1,0.500000,1.125000,0.280899,1.041667,8,10,18,267,...,CTX1,17,892,female,APP/PS1,poor,failed,optimal,optimal,False
9,9,DSB Iba1 17_40X_CTX2,0.560000,1.120000,0.332278,1.129032,25,25,50,316,...,CTX2,17,892,female,APP/PS1,poor,failed,optimal,optimal,False
